In [1]:
import os
import time
import shutil
from pathlib import Path

import pandas as pd
from tqdm import tqdm
from IPython.display import Audio, display
from mutagen.mp3 import MP3
from mutagen.wave import WAVE

from utils import download_audios, download_texts
from utils import usx_parser
from utils import audio_stats
# from utils import force_align_book

In [3]:
audio_df = audio_stats.get_all_audio_files(audios_dir="data/audios/Yoruba", alignment_filter="only")

total_hours_all_languages = audio_df["duration_seconds"].sum() / 3600
print(f"TOTAL HOURS (all languages): {total_hours_all_languages:.2f}")

audio_df.sort_values(by="duration_seconds", ascending=False)

TOTAL HOURS (all languages): 92.41


,language,testament_format,book,file_name,file_path,format,file_size_mb,duration_seconds
21628,Alignment,Jeremiah,JER_051_Verse_021.wav,JER_051_Verse_021.wav,data/audios/Yoruba/Alignment/Jeremiah/JER_051_...,wav,47.91,523.32
21630,Alignment,Jeremiah,JER_051_Verse_022.wav,JER_051_Verse_022.wav,data/audios/Yoruba/Alignment/Jeremiah/JER_051_...,wav,47.08,514.27
5157,Alignment,Luke,LUK_023_Verse_016.wav,LUK_023_Verse_016.wav,data/audios/Yoruba/Alignment/Luke/LUK_023_Vers...,wav,33.22,362.82
5159,Alignment,Luke,LUK_023_Verse_017.wav,LUK_023_Verse_017.wav,data/audios/Yoruba/Alignment/Luke/LUK_023_Vers...,wav,32.82,358.47
3794,Alignment,John,JHN_018_Verse_013.wav,JHN_018_Verse_013.wav,data/audios/Yoruba/Alignment/John/JHN_018_Vers...,wav,29.24,319.39
...,...,...,...,...,...,...,...,...
25223,Alignment,Leviticus,LEV_023_Verse_023.wav,LEV_023_Verse_023.wav,data/audios/Yoruba/Alignment/Leviticus/LEV_023...,wav,0.18,1.91
1962,Alignment,Acts,ACT_018_Verse_004.wav,ACT_018_Verse_004.wav,data/audios/Yoruba/Alignment/Acts/ACT_018_Vers...,wav,0.17,1.87
26935,Alignment,Numbers,NUM_026_Verse_052.wav,NUM_026_Verse_052.wav,data/audios/Yoruba/Alignment/Numbers/NUM_026_V...,wav,0.17,1.83
3608,Alignment,John,JHN_011_Verse_035.wav,JHN_011_Verse_035.wav,data/audios/Yoruba/Alignment/John/JHN_011_Vers...,wav,0.16,1.75


In [5]:
# Book to testament mapping (constant, defined outside function)
BOOK_TO_TESTAMENT = {
    # Old Testament
    "Genesis": "Old Testament",
    "Exodus": "Old Testament",
    "Leviticus": "Old Testament",
    "Numbers": "Old Testament",
    "Deuteronomy": "Old Testament",
    "Joshua": "Old Testament",
    "Judges": "Old Testament",
    "Ruth": "Old Testament",
    "1 Samuel": "Old Testament",
    "2 Samuel": "Old Testament",
    "1 Kings": "Old Testament",
    "2 Kings": "Old Testament",
    "1 Chronicles": "Old Testament",
    "2 Chronicles": "Old Testament",
    "Ezra": "Old Testament",
    "Nehemiah": "Old Testament",
    "Esther": "Old Testament",
    "Job": "Old Testament",
    "Psalms": "Old Testament",
    "Proverbs": "Old Testament",
    "Ecclesiastes": "Old Testament",
    "Song of Songs": "Old Testament",
    "Isaiah": "Old Testament",
    "Jeremiah": "Old Testament",
    "Lamentations": "Old Testament",
    "Ezekiel": "Old Testament",
    "Daniel": "Old Testament",
    "Hosea": "Old Testament",
    "Joel": "Old Testament",
    "Amos": "Old Testament",
    "Obadiah": "Old Testament",
    "Jonah": "Old Testament",
    "Micah": "Old Testament",
    "Nahum": "Old Testament",
    "Habakkuk": "Old Testament",
    "Zephaniah": "Old Testament",
    "Haggai": "Old Testament",
    "Zechariah": "Old Testament",
    "Malachi": "Old Testament",
    # New Testament
    "Matthew": "New Testament",
    "Mark": "New Testament",
    "Luke": "New Testament",
    "John": "New Testament",
    "Acts": "New Testament",
    "Romans": "New Testament",
    "1 Corinthians": "New Testament",
    "2 Corinthians": "New Testament",
    "Galatians": "New Testament",
    "Ephesians": "New Testament",
    "Philippians": "New Testament",
    "Colossians": "New Testament",
    "1 Thessalonians": "New Testament",
    "2 Thessalonians": "New Testament",
    "1 Timothy": "New Testament",
    "2 Timothy": "New Testament",
    "Titus": "New Testament",
    "Philemon": "New Testament",
    "Hebrews": "New Testament",
    "James": "New Testament",
    "1 Peter": "New Testament",
    "2 Peter": "New Testament",
    "1 John": "New Testament",
    "2 John": "New Testament",
    "3 John": "New Testament",
    "Jude": "New Testament",
    "Revelation": "New Testament",
}


def get_alignment_dataframe(language: str, base_dir: str = "data/audios") -> pd.DataFrame:
    """
    Load alignment data for a given language and return a DataFrame.
    
    Args:
        language: The language name (e.g., "Yoruba")
        base_dir: Base directory for audio files (default: "data/audios")
    
    Returns:
        DataFrame with columns: audio_file, text_file, text, book, chapter, 
                               verse, testament, duration_seconds
    """
    alignment_dir = os.path.join(base_dir, language, "Alignment")
    
    # Collect all audio files recursively (.wav and .mp3)
    audio_files = []
    for root, dirs, files in os.walk(alignment_dir):
        for file in files:
            if file.lower().endswith(('.wav', '.mp3')):
                audio_files.append(os.path.join(root, file))
    
    # Each audio file has a corresponding .txt file with the same name
    text_files = [os.path.splitext(x)[0] + ".txt" for x in audio_files]
    
    # Build initial dataframe with file paths
    df = pd.DataFrame({
        "audio_file": audio_files,
        "text_file": text_files,
    })
    
    # Helper to safely read text file contents
    def read_text_file(file_path):
        try:
            with open(file_path, "r", encoding="utf-8") as f:
                return f.read()
        except Exception:
            return None
    
    # Read transcript text from each text file
    df["text"] = df["text_file"].apply(read_text_file)
    
    # Extract metadata from file path structure:
    # Format: .../Alignment/{Book}/{BOOK_CHAPTER_Verse_VERSE}.txt
    df["book"] = df["text_file"].apply(lambda x: x.split("/")[-2])
    df["chapter"] = df["text_file"].apply(lambda x: x.replace(".txt", "").split("/")[-1].split("_")[1])
    df["verse"] = df["text_file"].apply(lambda x: x.replace(".txt", "").split("/")[-1].split("_")[-1])
    
    # Map book name to testament (Old/New)
    df["testament"] = df["book"].map(BOOK_TO_TESTAMENT)
    
    # Get audio duration in seconds
    df["duration_seconds"] = df["audio_file"].apply(audio_stats.get_audio_duration)

    # Reorder columns
    df = df[["audio_file", "text", "testament", "book", "chapter", "verse", "duration_seconds"]]
    
    return df

In [6]:
language = "Yoruba"
base_dir = "data/audios"

alignment_df = get_alignment_dataframe(language, base_dir)
alignment_df.head()

,audio_file,text,testament,book,chapter,verse,duration_seconds
0,data/audios/Yoruba/Alignment/1 Corinthians/1CO...,"Paulu, ẹni ti a pé láti jẹ́ aposteli Kristi Je...",New Testament,1 Corinthians,001,001,8.93
1,data/audios/Yoruba/Alignment/1 Corinthians/1CO...,"Sí ìjọ ènìyàn Ọlọ́run ni Kọrinti, sí àwọn ti a...",New Testament,1 Corinthians,001,002,17.72
2,data/audios/Yoruba/Alignment/1 Corinthians/1CO...,Oore-ọ̀fẹ́ àti àlàáfíà fún yín láti ọ̀dọ̀ Ọlọ́...,New Testament,1 Corinthians,001,003,8.90
3,data/audios/Yoruba/Alignment/1 Corinthians/1CO...,Nígbà gbogbo ni mo ń dúpẹ́ lọ́wọ́ Ọlọ́run fún ...,New Testament,1 Corinthians,001,004,6.91
4,data/audios/Yoruba/Alignment/1 Corinthians/1CO...,Nítorí nínú rẹ̀ ni a ti sọ yín di ọlọ́rọ̀ nínú...,New Testament,1 Corinthians,001,005,7.15


## Outlier removal

In [8]:
alignment_df_renamed = alignment_df.rename(columns={"audio_file": "wav_filename", "text": "transcript"})
alignment_df_renamed.to_csv("data-checker/files/Yoruba.csv")

In [13]:
alignment_df_renamed

,wav_filename,transcript,testament,book,chapter,verse,duration_seconds
0,data/audios/Yoruba/Alignment/1 Corinthians/1CO...,"Paulu, ẹni ti a pé láti jẹ́ aposteli Kristi Je...",New Testament,1 Corinthians,001,001,8.93
1,data/audios/Yoruba/Alignment/1 Corinthians/1CO...,"Sí ìjọ ènìyàn Ọlọ́run ni Kọrinti, sí àwọn ti a...",New Testament,1 Corinthians,001,002,17.72
2,data/audios/Yoruba/Alignment/1 Corinthians/1CO...,Oore-ọ̀fẹ́ àti àlàáfíà fún yín láti ọ̀dọ̀ Ọlọ́...,New Testament,1 Corinthians,001,003,8.90
3,data/audios/Yoruba/Alignment/1 Corinthians/1CO...,Nígbà gbogbo ni mo ń dúpẹ́ lọ́wọ́ Ọlọ́run fún ...,New Testament,1 Corinthians,001,004,6.91
4,data/audios/Yoruba/Alignment/1 Corinthians/1CO...,Nítorí nínú rẹ̀ ni a ti sọ yín di ọlọ́rọ̀ nínú...,New Testament,1 Corinthians,001,005,7.15
...,...,...,...,...,...,...,...
31095,data/audios/Yoruba/Alignment/Zephaniah/ZEP_003...,"Ní ọjọ́ náà, wọn yóò sọ fún Jerusalẹmu pé, “Má...",Old Testament,Zephaniah,003,016,8.22
31096,data/audios/Yoruba/Alignment/Zephaniah/ZEP_003...,"Olúwa Ọlọ́run rẹ wà pẹ̀lú rẹ, Ó ní agbára láti...",Old Testament,Zephaniah,003,017,14.61
31097,data/audios/Yoruba/Alignment/Zephaniah/ZEP_003...,“Èmi ó kó àwọn tí ó ń banújẹ́ fún àjọ̀dún tí a...,Old Testament,Zephaniah,003,018,8.69
31098,data/audios/Yoruba/Alignment/Zephaniah/ZEP_003...,Ní àkókò náà ni èmi yóò dojúkọ àwọn tí ń ni yí...,Old Testament,Zephaniah,003,019,14.33


In [ ]:
best = pd.read_csv("data-checker/files/Yoruba.BEST")
print(best.shape)

(29933, 16)


In [ ]:
python data-checker/data_checks.py data-checker/files/Yoruba.csv 2.0

## Upload to Hugging Face

In [ ]:
from datasets import Dataset, Audio
from huggingface_hub import HfApi

def upload_alignment_to_hf(
    alignment_df: pd.DataFrame,
    language: str,
    repo_id: str,
    private: bool = False,
    max_shard_size: str = "200MB",
    max_retries: int = 3,
):
    """
    Upload alignment data as a TTS dataset to Hugging Face Hub.
    
    Args:
        alignment_df: DataFrame with audio_file, text, and metadata columns
        language: Language name to use as the split/config name
        repo_id: Hugging Face repository ID (e.g., "username/bible-tts")
        private: Whether the dataset should be private
        max_shard_size: Maximum shard size for upload (smaller = more reliable)
        max_retries: Number of retries on timeout errors
    """
    # Create a copy to avoid modifying the original
    df = alignment_df.copy()
    
    # Rename audio_file to audio for HF convention
    df = df.rename(columns={"audio_file": "audio"})
    
    # Create HF Dataset
    dataset = Dataset.from_pandas(df, preserve_index=False)
    
    # Cast the audio column to Audio feature (this handles loading the actual audio files)
    dataset = dataset.cast_column("audio", Audio())
    
    # Push to hub with retry logic for timeout errors
    for attempt in range(max_retries):
        try:
            dataset.push_to_hub(
                repo_id=repo_id,
                config_name=language,  # Use language as the config name
                split="train",  # HF requires a split name, but you can ignore it when loading
                private=private,
                max_shard_size=max_shard_size,  # Smaller shards for more reliable uploads
            )
            print(f"✓ Uploaded {len(dataset)} samples for '{language}' to {repo_id}")
            return
        except Exception as e:
            if "timeout" in str(e).lower() or "ReadTimeout" in str(type(e).__name__):
                if attempt < max_retries - 1:
                    wait_time = (attempt + 1) * 30  # 30s, 60s, 90s
                    print(f"⚠ Timeout on attempt {attempt + 1}/{max_retries}. Retrying in {wait_time}s...")
                    time.sleep(wait_time)
                else:
                    print(f"✗ Failed after {max_retries} attempts. The data may have uploaded - check the repo.")
                    raise
            else:
                raise

/home/mila/g/guzmand/scratch/.conda/envs/ReadAlongs/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
LANGUAGES = [
    'Apali',
    'Arabic Standard',
    'Assamese',
    'Bengali',
    'Central Kurdish',
    'Chhattisgarhi',
    'Chichewa',
    'Dawro',
    'Dholuo',
    'Ewe',
    'Gamo',
    'Gofa',
    'Gujarati',
    'Haitian Creole',
    # 'Haryanvi',
    'Hausa',
    'Hiligaynon',
    'Hindi',
    'Igbo',
    'Kannada',
    'Kikuyu',
    'Lingala',
    'Luganda',
    'Malayalam',
    'Marathi',
    'Ndebele',
    'Oromo',
    'Punjabi',
    'Shona',
    'Swahili',
    'Tamil',
    'Telugu',
    # 'Toma',
    'Turkish',
    'Twi (Akuapem)',
    'Twi (Asante)',
    'Ukrainian',
    'Urdu',
    'Vietnamese',
    'Yoruba'
]
base_dir = "data/audios"

for language in tqdm(LANGUAGES, desc="Uploading languages"):
    print(f"\n--- Processing '{language}' ---")
    try:
        alignment_df = get_alignment_dataframe(language, base_dir)

        # Simple outlier removal, need to do better
        alignment_df = alignment_df[alignment_df["duration_seconds"] < 60]

        repo_id = "davidguzmanr/bible-tts-resources"  # Change this to your repo ID

        upload_alignment_to_hf(
            alignment_df=alignment_df.head(100),
            language=language,
            repo_id=repo_id,
            private=False,
        )
    except Exception as e:
        print(f"✗ Error uploading '{language}': {e}")
    else:
        print(f"✓ Finished uploading '{language}'")


Uploading languages:   0%|          | 0/38 [00:00<?, ?it/s]


--- Processing 'Apali' ---




Map: 100%|██████████| 100/100 [00:00<00:00, 232.01 examples/s]


Creating parquet from Arrow format: 100%|██████████| 2/2 [00:00<00:00,  7.62ba/s]
Processing Files (1 / 1): 100%|██████████|  125MB /  125MB, 56.8MB/s  
New Data Upload: 100%|██████████|  125MB /  125MB, 56.8MB/s  
Uploading languages:   3%|▎         | 1/38 [00:35<21:40, 35.14s/it]

✓ Uploaded 100 samples for 'Apali' to davidguzmanr/bible-tts-resources
✓ Finished uploading 'Apali'

--- Processing 'Arabic Standard' ---




Map: 100%|██████████| 100/100 [00:00<00:00, 478.27 examples/s]

Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 16.49ba/s]
Processing Files (1 / 1): 100%|██████████| 37.5MB / 37.5MB, 37.4MB/s  
New Data Upload: 100%|██████████| 37.5MB / 37.5MB, 37.4MB/s  
Uploading languages:   5%|▌         | 2/38 [01:46<33:53, 56.49s/it]

...

✓ Uploaded 100 samples for 'Ndebele' to davidguzmanr/bible-tts-resources
✓ Finished uploading 'Ndebele'

--- Processing 'Oromo' ---




Map: 100%|██████████| 100/100 [00:00<00:00, 239.76 examples/s]


Creating parquet from Arrow format: 100%|██████████| 2/2 [00:00<00:00, 11.04ba/s]
Processing Files (1 / 1): 100%|██████████| 84.8MB / 84.8MB, 47.1MB/s  
New Data Upload: 100%|██████████| 84.8MB / 84.8MB, 47.1MB/s  
Uploading languages:  68%|██████▊   | 26/38 [31:55<15:32, 77.72s/it]

✓ Uploaded 100 samples for 'Oromo' to davidguzmanr/bible-tts-resources
✓ Finished uploading 'Oromo'

--- Processing 'Punjabi' ---




Map: 100%|██████████| 100/100 [00:00<00:00, 234.73 examples/s]


Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00,  4.40ba/s]
Processing Files (1 / 1): 100%|██████████| 85.1MB / 85.1MB, 42.5MB/s  
New Data Upload: 100%|██████████| 85.1MB / 85.1MB, 42.5MB/s  
Uploading languages:  71%|███████   | 27/38 [33:11<14:10, 77.31s/it]

✓ Uploaded 100 samples for 'Punjabi' to davidguzmanr/bible-tts-resources
✓ Finished uploading 'Punjabi'

--- Processing 'Shona' ---


In [8]:
# Upload the dataset to Hugging Face
# Make sure you're logged in: huggingface-cli login

repo_id = "davidguzmanr/bible-tts-resources"  # Change this to your repo ID

upload_alignment_to_hf(
    alignment_df=alignment_df.head(100),
    language=language,  # "Yoruba"
    repo_id=repo_id,
    private=False,
)

Creating parquet from Arrow format: 100%|██████████| 2/2 [00:00<00:00, 13.68ba/s]
Processing Files (1 / 1): 100%|██████████| 80.0MB / 80.0MB, 44.5MB/s  
New Data Upload: 100%|██████████| 80.0MB / 80.0MB, 44.5MB/s  
Uploading the dataset shards: 100%|██████████| 1/1 [00:05<00:00,  5.02s/ shards]


ReadTimeout: The read operation timed out

In [ ]:
# To load the dataset later, use:
# from datasets import load_dataset
# ds = load_dataset("YOUR_USERNAME/bible-tts", "Yoruba")  # Load specific language
# ds = load_dataset("YOUR_USERNAME/bible-tts", "Hausa")   # Load another language